![sql](images/sql-logo.jpg)

## Objectives

- Use SQL aggregation functions with GROUP BY
- Use HAVING for group filtering
- Use SQL JOIN to combine tables using keys

In [1]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("data/flights.db")
cur = conn.cursor()

## A Quick Note about Execution Order!

![query execution order, from https://wizardzines.com/comics/sql-query-order/](images/sql-query-order.png)

[[Image Source]](https://wizardzines.com/comics/sql-query-order/)

# Aggregating Functions

>  A SQL **aggregating function** takes in many values and returns one value.

We have already seen some SQL aggregating functions like `COUNT()`. There are also others, like SUM(), AVG(), MIN(), and MAX().

### Example Simple Aggregations

Find the max value for `longitude` in the `airports` table:

In [16]:
query = '''
    SELECT MAX(
        CAST(longitude AS REAL)
        ) AS "Max Longitude"
    FROM airports
'''

pd.read_sql(query, conn)

,Max Longitude
0,179.951


Find the max value for `id` in the `airports` table:

In [19]:
query = '''
    SELECT MAX(CAST(id AS INT)) AS "Biggest ID"
    FROM airports;
'''

pd.read_sql(query, conn)

,Biggest ID
0,9541


Find the count for all inactive airlines:

In [25]:
query = '''
    SELECT COUNT(*)
    FROM airlines
    WHERE active IS "N";
'''

pd.read_sql(query, conn)

,COUNT(*)
0,4886


## Grouping in SQL

We can go deeper and use aggregation functions on _groups_ using the `GROUP BY` clause.

The `GROUP BY` clause will group one or more columns together to perform aggregation functions on.

## Example `GROUP BY`  Statements

Let's say we want to know how many active and non-active airlines there are.

### Without `GROUP BY`

Let's first start with just seeing how many airlines there are:

In [26]:
query = '''
    SELECT 
        COUNT() AS "Number of Airlines"
    FROM 
        airlines
'''

pd.read_sql(query, conn)

,Number of Airlines
0,6048


One way for us to get the counts for each is to create two queries that will filter each kind of airline (active vs non-active) and count those values:

In [27]:
active_query = '''
    SELECT 
        COUNT() AS "Number of Active Airlines"
    FROM 
        airlines
    WHERE 
        active='Y'
'''

not_active_query = '''
    SELECT 
        COUNT() AS "Number of Non Active Airlines"
    FROM 
        airlines
    WHERE 
        active='N'
'''

display(pd.read_sql(active_query, conn))
display(pd.read_sql(not_active_query, conn))

,Number of Active Airlines
0,1161


,Number of Non Active Airlines
0,4886


This works but it's inefficient.

### With `GROUP BY`

Instead, we can tell the SQL server to do the work for us by grouping values we care about for us!

In [28]:
query = '''
    SELECT 
        COUNT() AS number_of_airlines
    FROM 
        airlines
    GROUP BY
        active
'''

pd.read_sql(query, conn)

,number_of_airlines
0,4886
1,1161
2,1


This is great! And if you look closely, you can observe we have _three_ different groups instead of our expected two!

Let's also print out the `active` column values for each group/aggregation so we know what we're looking at:

In [32]:
query = '''
    SELECT 
        active,
        COUNT() AS number_of_airlines
    FROM 
        airlines
    GROUP BY
        UPPER(active)
'''

pd.read_sql(query, conn)

,active,number_of_airlines
0,N,4887
1,Y,1161


What do we think this extra category captures? Can we filter those out?

## Exercises

### Question 1:

Which countries have the highest numbers of active airlines? Return the top 10.

> Note that the `GROUP BY` clause is considered _before_ the `ORDER BY` and `LIMIT` clauses

In [50]:
query = '''
    SELECT country, COUNT(*) AS "Number of Airlines"
    FROM airlines
    WHERE active IS "Y"
    GROUP BY country
    ORDER BY "Number of Airlines" DESC
    LIMIT 10;
'''

pd.read_sql(query, conn)

,country,Number of Airlines
0,United States,141
1,Russia,72
2,United Kingdom,40
3,Germany,37
4,Canada,34
5,Australia,26
6,China,25
7,Spain,24
8,Brazil,23
9,France,22


In [53]:
df = pd.read_sql("SELECT * FROM airlines", conn)
df.loc[df['active'] == 'Y']['country'].value_counts()

United States            141
Russia                    72
United Kingdom            40
Germany                   37
Canada                    34
                        ... 
Namibia                    1
Cayman Islands             1
Trinidad and Tobago        1
Republic of the Congo      1
Puerto Rico                1
Name: country, Length: 194, dtype: int64

### Question 2:

How many airports are in each time zone?

In [51]:
query = '''
    SELECT timezone, COUNT(timezone) AS Count
    FROM airports
    GROUP BY timezone;
'''

pd.read_sql(query, conn)

,timezone,Count
0,Africa/Abidjan,7
1,Africa/Accra,6
2,Africa/Addis_Ababa,31
3,Africa/Algiers,44
4,Africa/Asmera,4
...,...,...
289,Pacific/Tongatapu,6
290,Pacific/Truk,3
291,Pacific/Wake,1
292,Pacific/Wallis,2


## Filtering Groups with `HAVING`

We showed that you can filter tables with `WHERE`. We can similarly filter _groups/aggregations_ using `HAVING` clauses.

## Examples of  `HAVING`

### Simple Filtering - Number of Airports in a Country

Let's come back to the aggregation of active airports:

In [54]:
query = '''
    SELECT 
        COUNT() AS num,
        country
    FROM 
        airlines
    WHERE 
        active='Y'
    GROUP BY 
        country
    ORDER BY 
        num DESC
'''

pd.read_sql(query, conn)

,num,country
0,141,United States
1,72,Russia
2,40,United Kingdom
3,37,Germany
4,34,Canada
...,...,...
190,1,Antigua and Barbuda
191,1,American Samoa
192,1,AVIANCA
193,1,ALASKA


We can see we have a lot of results. But maybe we only want to keep the countries that have more than $30$ active airlines:

In [63]:
query = '''
    SELECT 
        country,
        COUNT() AS num
    FROM 
        airlines
    WHERE 
        active='Y'
    GROUP BY 
        country
    HAVING
        num > 30
    ORDER BY 
        num DESC
'''

pd.read_sql(query, conn)

,country,num
0,United States,141
1,Russia,72
2,United Kingdom,40
3,Germany,37
4,Canada,34


## Filtering Different Aggregations - Airport Altitudes

We can also filter on other aggregations. For example, let's say we want to investigate the `airports` table.

Specifically, we want to know the height of the _highest airport_ in a country given that it has _at least $100$ airports_.

### Looking at the `airports` Table

In [64]:
query = '''
    SELECT 
        *
    FROM 
        airports 
'''
pd.read_sql(query, conn).head()

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby
2,2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10,U,Pacific/Port_Moresby
3,3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10,U,Pacific/Port_Moresby
4,4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.22005,146,10,U,Pacific/Port_Moresby


### Looking at the Highest Airport

Let's first get the highest altitude for each airport:

In [65]:
query = '''
    SELECT 
        country,
        MAX(CAST(altitude AS int)) AS highest_airport_in_country
    FROM 
        airports 
    GROUP BY
        country
    ORDER BY
        country
'''

pd.read_sql(query, conn)

,country,highest_airport_in_country
0,Afghanistan,7400
1,Albania,126
2,Algeria,4518
3,American Samoa,110
4,Angola,5778
...,...,...
235,West Bank,2485
236,Western Sahara,350
237,Yemen,7216
238,Zambia,4636


### Looking at the Number of Airports Too

We can also get the number of airports for each country.

In [66]:
query = '''
    SELECT 
        country,
        MAX(CAST(altitude AS int)) AS highest_airport_in_country,
        COUNT() AS number_of_airports_in_country
    FROM
        airports 
    GROUP BY
        country
    ORDER BY
        country
'''

pd.read_sql(query, conn)

,country,highest_airport_in_country,number_of_airports_in_country
0,Afghanistan,7400,21
1,Albania,126,1
2,Algeria,4518,44
3,American Samoa,110,3
4,Angola,5778,26
...,...,...,...
235,West Bank,2485,1
236,Western Sahara,350,3
237,Yemen,7216,11
238,Zambia,4636,13


### Filtering on Aggregations

> Recall:
>
> We want to know the height of the _highest airport_ in a country given that it has _at least $100$ airports_.

In [76]:
query = '''
    SELECT 
        country,
        MAX(CAST(altitude AS int)) AS highest_airport_in_country
        -- Note we don't have to include this in our SELECT to use it to filter!
        --,COUNT() AS number_of_airports_in_country
    FROM
        airports 
    GROUP BY
        country
    HAVING
        COUNT() >= 100
    ORDER BY
        country
'''

pd.read_sql(query, conn)

,country,highest_airport_in_country
0,Argentina,11414
1,Australia,4260
2,Brazil,8708
3,Canada,3939
4,China,14472
5,France,6588
6,Germany,2297
7,India,10682
8,Indonesia,9104
9,Japan,2200


# Joins

The biggest advantage in using a relational database (like we've been with SQL) is that you can create **joins**.

> By using **`JOIN`** in our query, we can connect different tables using their _relationships_ to other tables.
>
> Usually we use a key (*foreign key*) to tell us how the two tables are related.

There are different types of joins and each has their different use case - because SQL joins can be used to both **add** data to a table and **remove** data from a table. 

![venn](images/venn.png)

## `INNER JOIN`

> An **inner join** will join two tables together and only keep rows if the _key is in both tables_

![](images/inner_join.png)

Example of an inner join:

```sql
SELECT
    table1.column_name,
    table2.different_column_name
FROM
    table1
    INNER JOIN table2
        ON table1.shared_column_name = table2.shared_column_name
```

### Code Example for Inner Joins

Let's say we want to look at the different airplane routes

In [77]:
query = '''
    SELECT 
        *
    FROM
        routes 
'''

pd.read_sql(query, conn)

,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment
0,0,2B,410,AER,2965,KZN,2990,None,0,CR2
1,1,2B,410,ASF,2966,KZN,2990,None,0,CR2
2,2,2B,410,ASF,2966,MRV,2962,None,0,CR2
3,3,2B,410,CEK,2968,KZN,2990,None,0,CR2
4,4,2B,410,CEK,2968,OVB,4078,None,0,CR2
...,...,...,...,...,...,...,...,...,...,...
67658,67658,ZL,4178,WYA,6334,ADL,3341,None,0,SF3
67659,67659,ZM,19016,DME,4029,FRU,2912,None,0,734
67660,67660,ZM,19016,FRU,2912,DME,4029,None,0,734
67661,67661,ZM,19016,FRU,2912,OSS,2913,None,0,734


This is great but notice the `airline_id` column. It'd be nice to have some more information about the airlines associated with these routes.

We can do an **inner join** to get this information!

#### Inner Join Routes & Airline Data

In [78]:
query = '''
    SELECT 
        *
    FROM
        routes
    INNER JOIN airlines
        ON routes.airline_id = airlines.id
'''

pd.read_sql(query, conn)

,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment,index,id,name,alias,iata,icao,callsign,country,active
0,0,2B,410,AER,2965,KZN,2990,None,0,CR2,409,410,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
1,1,2B,410,ASF,2966,KZN,2990,None,0,CR2,409,410,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
2,2,2B,410,ASF,2966,MRV,2962,None,0,CR2,409,410,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
3,3,2B,410,CEK,2968,KZN,2990,None,0,CR2,409,410,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
4,4,2B,410,CEK,2968,OVB,4078,None,0,CR2,409,410,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66980,67658,ZL,4178,WYA,6334,ADL,3341,None,0,SF3,4171,4178,Regional Express,\N,ZL,RXA,REX,Australia,Y
66981,67659,ZM,19016,DME,4029,FRU,2912,None,0,734,5977,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,Y
66982,67660,ZM,19016,FRU,2912,DME,4029,None,0,734,5977,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,Y
66983,67661,ZM,19016,FRU,2912,OSS,2913,None,0,734,5977,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,Y


We can also specify that we want to retain only certain columns in the `SELECT` clause:

In [79]:
query = '''
    SELECT 
        routes.source AS departing,
        routes.dest AS destination,
        routes.stops AS stops_before_destination,
        airlines.name AS airline_name
    FROM
        routes
        INNER JOIN airlines
            ON routes.airline_id = airlines.id
'''

pd.read_sql(query, conn)

,departing,destination,stops_before_destination,airline_name
0,AER,KZN,0,Aerocondor
1,ASF,KZN,0,Aerocondor
2,ASF,MRV,0,Aerocondor
3,CEK,KZN,0,Aerocondor
4,CEK,OVB,0,Aerocondor
...,...,...,...,...
66980,WYA,ADL,0,Regional Express
66981,DME,FRU,0,Apache Air
66982,FRU,DME,0,Apache Air
66983,FRU,OSS,0,Apache Air


Also we can alias the different tables to make the queries a bit easier to write!

In [81]:
query = '''
    SELECT 
        r.source AS departing,
        r.dest AS destination,
        r.stops AS stops_before_destination,
        al.name AS airline_name
    FROM
        routes AS r
        INNER JOIN airlines AS al
            ON r.airline_id = al.id
'''

pd.read_sql(query, conn)

,departing,destination,stops_before_destination,airline_name
0,AER,KZN,0,Aerocondor
1,ASF,KZN,0,Aerocondor
2,ASF,MRV,0,Aerocondor
3,CEK,KZN,0,Aerocondor
4,CEK,OVB,0,Aerocondor
...,...,...,...,...
66980,WYA,ADL,0,Regional Express
66981,DME,FRU,0,Apache Air
66982,FRU,DME,0,Apache Air
66983,FRU,OSS,0,Apache Air


#### Note: Losing Data with Inner Joins

Since data rows are kept only if _both_ tables have the key, some data can be lost

In [82]:
df_all_routes = pd.read_sql('''
    SELECT 
        *
    FROM
        routes
''', conn)

df_routes_after_join = pd.read_sql('''
    SELECT 
        *
    FROM
        routes
        INNER JOIN airlines
            ON routes.airline_id = airlines.id
''', conn)

In [83]:
# Look at how the number of rows are different
df_all_routes.shape, df_routes_after_join.shape

((67663, 10), (66985, 19))

If you want to keep your data from at least one of your tables, you should use a left join instead of an inner join.

## `LEFT JOIN`

> A **left join** will join two tables together and but will keep all data from the first (left) table using the key provided.

![](images/left_join.png)

Example of a left and right join:

```sql
SELECT
    table1.column_name,
    table2.different_column_name
FROM
    table1
    LEFT JOIN table2
        ON table1.shared_column_name = table2.shared_column_name
```

### Code Example for Left Join

If wanted to ensure we always had every route even if the key in `airlines` was not found, we could replace our `INNER JOIN` with a `LEFT JOIN`:

In [84]:
# This will include all the data from routes
df_routes_after_left_join = pd.read_sql('''
    SELECT 
        *
    FROM
        routes
        LEFT JOIN airlines
            ON routes.airline_id = airlines.id
''', conn)

df_routes_after_left_join.shape

(67663, 19)

## Exercise: 

Which airline has the most routes listed in our database?

In [85]:
pd.read_sql("SELECT * FROM airlines LIMIT 3;", conn)

,index,id,name,alias,iata,icao,callsign,country,active
0,0,1,Private flight,\N,-,None,None,None,Y
1,1,2,135 Airways,\N,None,GNL,GENERAL,United States,N
2,2,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y


In [86]:
pd.read_sql("SELECT * FROM routes LIMIT 3;", conn)

,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment
0,0,2B,410,AER,2965,KZN,2990,None,0,CR2
1,1,2B,410,ASF,2966,KZN,2990,None,0,CR2
2,2,2B,410,ASF,2966,MRV,2962,None,0,CR2


In [102]:
query = '''
    SELECT 
        al.name,
        COUNT() AS number_of_routes
    FROM airlines AS al
    INNER JOIN routes AS r
        ON r.airline_id = al.id
    GROUP BY al.name
    ORDER BY number_of_routes DESC
    LIMIT 1;
'''

pd.read_sql(query, conn)

,name,number_of_routes
0,Ryanair,2484
